In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-c-n-siddhi/Dataset_SIDDHI/NORMAL/20240522124326_left_anterior9344.png
/kaggle/input/dataset-c-n-siddhi/Dataset_SIDDHI/NORMAL/20240404105501_right_anterior6772.png
/kaggle/input/dataset-c-n-siddhi/Dataset_SIDDHI/NORMAL/20240306103946_right_anterior6838.png
/kaggle/input/dataset-c-n-siddhi/Dataset_SIDDHI/NORMAL/20240305113045_right_anterior1.png
/kaggle/input/dataset-c-n-siddhi/Dataset_SIDDHI/NORMAL/20240405102147_right_anterior6647.png
/kaggle/input/dataset-c-n-siddhi/Dataset_SIDDHI/NORMAL/20240307104417_right_anterior3714.png
/kaggle/input/dataset-c-n-siddhi/Dataset_SIDDHI/NORMAL/20240301113346_left_anterior4508.png
/kaggle/input/dataset-c-n-siddhi/Dataset_SIDDHI/NORMAL/20240321110856_right_anterior7097.png
/kaggle/input/dataset-c-n-siddhi/Dataset_SIDDHI/NORMAL/20240307105553_right_anterior8412.png
/kaggle/input/dataset-c-n-siddhi/Dataset_SIDDHI/NORMAL/20240326115815_right_anterior3592.png
/kaggle/input/dataset-c-n-siddhi/Dataset_SIDDHI/NORMAL/20240410110730_left_

In [2]:
from transformers import AutoImageProcessor, ConvNextV2ForImageClassification
import torch
from datasets import load_dataset

2024-06-30 17:24:02.406092: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 17:24:02.406198: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 17:24:02.521143: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
preprocessor = AutoImageProcessor.from_pretrained("facebook/convnextv2-large-22k-384")
model = ConvNextV2ForImageClassification.from_pretrained("facebook/convnextv2-huge-22k-384")

preprocessor_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.64G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
for param in model.parameters():
    param.requires_grad = False

# Replace or add new layers
# Example: Replace classifier with a new one suitable for binary classification
in_features = model.classifier.in_features
# Example: Adding additional fully connected layers
model.classifier = torch.nn.Sequential(
    torch.nn.Linear(in_features, 512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
#     torch.nn.Dropout(0.5),
    torch.nn.Linear(256, 2)  # Output layer for binary classification
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # Example scheduler

criterion = torch.nn.CrossEntropyLoss()

In [5]:
from PIL import Image
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split


dataset_path = r"/kaggle/input/dataset-c-n-siddhi/Dataset_SIDDHI"


class CropAndTransform:
    def __init__(self, crop_box, transform):
        self.crop_box = crop_box  # (left, upper, right, lower)
        self.transform = transform

    def __call__(self, image):
        image = image.crop(self.crop_box)
        return self.transform(image)


crop_box = (450, 350, 1400, 1200)  # (left, upper, right, lower) to crop to 1360 x 540


transform = transforms.Compose([
    CropAndTransform(crop_box, transforms.Resize((384, 384))),
#     transforms.Resize((384, 384)),
#     transforms.RandomRotation(20),
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the dataset with the custom transform
dataset = ImageFolder(root=dataset_path, transform=transform)
# path2=r'/kaggle/input/cataract-image-dataset/processed_images/test'
# test_dataset=ImageFolder(root=path2, transform=transform)

# Split the dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,num_workers=4)

In [ ]:
# Training loop
num_epochs=20
for epoch in range(20):
    model.train()
    running_loss = 0.0
    kkk = 0
    
    for inputs, labels in train_loader:
        kkk += 1
        print(f"Epoch {epoch+1}, Batch {kkk}")
        
        # Move inputs and labels to GPU
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(pixel_values=inputs).logits
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Accumulate loss
        running_loss += loss.item()
    
    # Evaluate on validation set
    model.eval()
    total = 0
    correct = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(pixel_values=inputs).logits
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        epoch_accuracy = correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}, Accuracy: {epoch_accuracy * 100}%")
        
        
        scheduler.step()

# Save the final trained model weights
final_model_path = "final_model.pt"
torch.save(model.state_dict(), final_model_path)
print(f"Final model saved to {final_model_path}")

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1, Batch 1
Epoch 1, Batch 2
Epoch 1, Batch 3
Epoch 1, Batch 4
Epoch 1, Batch 5
Epoch 1, Batch 6
Epoch 1, Batch 7
Epoch 1, Batch 8
Epoch 1, Batch 9
Epoch 1, Batch 10
Epoch 1, Batch 11
Epoch 1, Batch 12
Epoch 1, Batch 13
Epoch 1, Batch 14
Epoch 1, Batch 15
Epoch 1, Batch 16
Epoch 1, Batch 17
Epoch 1, Batch 18
Epoch 1, Batch 19
Epoch 1, Batch 20
Epoch 1, Batch 21
Epoch 1, Batch 22
Epoch 1, Batch 23
Epoch 1, Batch 24
Epoch 1, Batch 25
Epoch 1, Batch 26
Epoch 1, Batch 27
Epoch 1, Batch 28
Epoch 1, Batch 29
Epoch 1, Batch 30
Epoch 1, Batch 31
Epoch 1, Batch 32
Epoch 1, Batch 33
Epoch 1, Batch 34
Epoch 1, Batch 35
Epoch 1, Batch 36
Epoch 1, Batch 37
Epoch 1, Batch 38
Epoch 1, Batch 39
Epoch 1, Batch 40
Epoch 1, Batch 41
Epoch 1, Batch 42
Epoch 1, Batch 43
Epoch 1, Batch 44
Epoch 1, Batch 45
Epoch 1, Batch 46
Epoch 1, Batch 47
Epoch 1, Batch 48
Epoch 1, Batch 49
Epoch 1, Batch 50
Epoch 1, Batch 51
Epoch 1, Batch 52
Epoch 1, Batch 53
Epoch 1, Batch 54
Epoch 1, Batch 55
Epoch 1, Batch 56
E

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 1/20, Loss: 0.6470352092054155, Accuracy: 61.86770428015564%
Epoch 2, Batch 1
Epoch 2, Batch 2
Epoch 2, Batch 3
Epoch 2, Batch 4
Epoch 2, Batch 5
Epoch 2, Batch 6
Epoch 2, Batch 7
Epoch 2, Batch 8
Epoch 2, Batch 9
Epoch 2, Batch 10
Epoch 2, Batch 11
Epoch 2, Batch 12
Epoch 2, Batch 13
Epoch 2, Batch 14
Epoch 2, Batch 15
Epoch 2, Batch 16
Epoch 2, Batch 17
Epoch 2, Batch 18
Epoch 2, Batch 19
Epoch 2, Batch 20
Epoch 2, Batch 21
Epoch 2, Batch 22
Epoch 2, Batch 23
Epoch 2, Batch 24
Epoch 2, Batch 25
Epoch 2, Batch 26
Epoch 2, Batch 27
Epoch 2, Batch 28
Epoch 2, Batch 29
Epoch 2, Batch 30
Epoch 2, Batch 31
Epoch 2, Batch 32
Epoch 2, Batch 33
Epoch 2, Batch 34
Epoch 2, Batch 35
Epoch 2, Batch 36
Epoch 2, Batch 37
Epoch 2, Batch 38
Epoch 2, Batch 39
Epoch 2, Batch 40
Epoch 2, Batch 41
Epoch 2, Batch 42
Epoch 2, Batch 43
Epoch 2, Batch 44
Epoch 2, Batch 45
Epoch 2, Batch 46
Epoch 2, Batch 47
Epoch 2, Batch 48
Epoch 2, Batch 49
Epoch 2, Batch 50
Epoch 2, Batch 51
Epoch 2, Batch 52
Epoch 